In [1]:
import pandas as pd
import gzip
from protein_clip import get_model, embed_sequence, batch_embed_sequence
from tqdm import tqdm

In [2]:
df = pd.read_parquet("./uniprot-esm2-layers6.parquet")

In [7]:
# I want to run all the sequences through the esm model. Which size?
layer = 6
model, alphabet = get_model(layer)
model

ESM2(
  (embed_tokens): Embedding(33, 320, padding_idx=1)
  (layers): ModuleList(
    (0-5): 6 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=320, out_features=320, bias=True)
        (v_proj): Linear(in_features=320, out_features=320, bias=True)
        (q_proj): Linear(in_features=320, out_features=320, bias=True)
        (out_proj): Linear(in_features=320, out_features=320, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=320, out_features=1280, bias=True)
      (fc2): Linear(in_features=1280, out_features=320, bias=True)
      (final_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=120, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((320,), eps=1e-05, elementwis

In [8]:
checkpoint_every = 1_000
not_computed_yet = df[df["embedding"].apply(len) == 1]
j = 0
for i, row in tqdm(not_computed_yet.iterrows(), total=len(not_computed_yet)):
    embed = embed_sequence(row["sequence"], model, alphabet, layer)
    df.at[i, "embedding"] = embed
    
    # save to disk every once and a while
    if j > checkpoint_every:
        df.to_parquet(f"uniprot-esm2-layers{layer}.parquet")
        j = 0
    j += 1

  0%|          | 523/461581 [02:07<31:20:09,  4.09it/s] 


KeyboardInterrupt: 